In [12]:
%%writefile train/lightgbm.py
from rest_api import serve

def serve():
    return serve()


def meow():
    print("meow meow cats")
    
    
if __name__ == '__main__':
    meow()

Overwriting train/lightgbm.py


In [4]:
%%writefile train/rest_api.py

def serve():
    print('sdcfghnrfvbhtgfdgbf')


Writing train/rest_api.py


In [9]:
%%writefile train/__init__.py


Writing train/__init__.py
